# Flex flow as evaluator
In this notebook we will demonstrate saving evaluator to the container registry, and using saved flex flow evaluators.

### Import

In [ ]:
import os
import json
import pandas as pd

from azure.ai.ml import MLClient
from azure.identity._credentials.default import AzureCliCredential
from azure.ai.ml.entities import Model

from promptflow.client import PFClient, load_flow
from promptflow.core import Flow
from promptflow.evals.evaluate import evaluate
from promptflow.evals.evaluators import F1ScoreEvaluator

## Save and uploading of the evaluator

In [ ]:
pf = PFClient()
pf.flows.save(F1ScoreEvaluator, path="./f1_score")

### Authenticate to azure
First we will need to authenticate to azure. For this purpose we will use the the configuration file of the net structure.
```json
{
    "resource_group_name": "resource-group-namee",
    "subscription_id": "subscription-uuid",
    "registry_name": "registry-name"
}
```
**Note:** If the `registry_name` will be replaced by `workspace_name`, the evaluator will be saved to Azure ML Workspace instead of registry.


In [ ]:
with open("config.json") as f:
    configuration = json.load(f)

Upload the saved evaluator.

In [ ]:
credential = AzureCliCredential()
ml_client = MLClient(credential=credential, **configuration)

eval = Model(
    path="f1_score",
    name="f1_score_uploaded",
    description="F1 score evaluator.",
)
ml_client.evaluators.create_or_update(eval)

Download the evaluator.

In [ ]:
ml_client.evaluators.download("f1_score_uploaded", version="1", download_path=".")

## Data
Now let us generate some data for testing.

In [ ]:
data_raw = pd.DataFrame(
    {
        "question": ["What is the name of Alexander the Great horse?", "What is Sun?"],
        "answer": ["Binkey", "The star."],
        "ground_truth": ["Bucephalus", "The star."],
    }
)
data_raw.to_json("data.jsonl", lines=True, orient="records")

## Test the downloaded evaluator
We will load evaluator in two ways:
- Using `Flow.load` method;
- Using `promptflow.client.load_flow` function.

### Run using `Flow.load`
<font color='red'> This section is commented out, because it used to fail </font><br>
First we will try using flow as a function.

In [ ]:
flow_path = os.path.join("f1_score_uploaded", "f1_score")
# downloaded_flow = Flow.load(flow_path)
# downloaded_flow(**data_raw[:1].T[0].to_dict())

Now we will run the evaluation using `evaluate` API.

In [ ]:
# results = evaluate(
#     data='data.jsonl',
#     evaluators = {'prompty_eval': downloaded_flow}
# )

### Run using `promptflow.client.load_flow`

In [ ]:
downloaded_flow = load_flow(flow_path)
# FlexFlow can not be called as a function.
# downloaded_flow(**data_raw[:1].T[0].to_dict())

In [ ]:
results = evaluate(data="data.jsonl", evaluators={"prompty_eval": downloaded_flow})

View results

In [ ]:
print(f"{results['metrics']=}")
pd.DataFrame(results["rows"])